In [4]:
!pip install SentencePiece

     |████████████████████████████████| 7.1MB 9.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 8.2MB/s  eta 0:00:01
     |████████████████████████████████| 17.3MB 86.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 5.3MB/s  eta 0:00:01
     |████████████████████████████████| 225kB 101.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 6.1MB/s  eta 0:00:01
     |████████████████████████████████| 7.8MB 85.9MB/s eta 0:00:01
     |████████████████████████████████| 706kB 89.9MB/s eta 0:00:01
     |████████████████████████████████| 778kB 99.9MB/s eta 0:00:01
     |████████████████████████████████| 163kB 108.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.2MB/s  eta 0:00:01
     |████████████████████████████████| 163kB 101.6MB/s eta 0:00:01
     |████████████████████████████████| 133kB 103.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 101.4MB/s eta 0:00:01
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, 

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     |████████████████████████████████| 195.4MB 32kB/s /s eta 0:00:01
     |████████████████████████████████| 24.9MB 102.6MB/s eta 0:00:01
     |████████████████████████████████| 4.1MB 106.7MB/s eta 0:00:01
     |████████████████████████████████| 2.0MB 114.3MB/s eta 0:00:01
     |████████████████████████████████| 6.5MB 97.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 124.4MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 109.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 3.6MB/s s eta 0:00:01
     |████████████████████████████████| 3.2MB 105.8MB/s eta 0:00:01
     |████████████████████████████████| 542kB 113.9MB/s eta 0:00:01
     |████████████████████████████████| 163kB 111.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 114.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.9MB/s s eta 0:00:01
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.

In [1]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

## Use GPU (cuda device) if avalable to Train the LLM

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
device

'cpu'

## Custom dataset for summarization
use following constructor to pass dataframe, tokenizer, maximum length of source text and maximum length of summary length

In [15]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

## Creating the training function
 The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 


In [16]:

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            print({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [17]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [24]:
# import pickle
# import os
# tup = ("hello","54","32cdsa")
# filename = '/data/artifacts/sample.pkl'
# os.makedirs(os.path.dirname(filename), exist_ok=True)
# with open(filename, 'wb') as f:
#     pickle.dump(tup, f)


In [25]:
# with open('/data/artifacts/sample.pkl', 'rb') as f:
#      d = pickle.load(f)

In [18]:
# Defining some key variables that will be used later on in the training  
TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 2       # number of epochs to train (default: 10)
VAL_EPOCHS = 1 
LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
SEED = 42               # random seed (default: 42)
MAX_LEN = 512
SUMMARY_LEN = 150 
# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(SEED) # pytorch random seed
np.random.seed(SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

In [3]:

# tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/packages/Python/c3e3a6d1-48c5-4ec2-8739-9b9b511c7036/3.8/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [20]:
# Importing and Pre-Processing the domain data
# Selecting the needed columns only. 
# Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
df = pd.read_csv('/data/news_summary.csv',encoding='latin-1')
df = df[['text','ctext']]
df.ctext = 'summarize: ' + df.ctext
print(df.head())

                                                text  \
0  The Administration of Union Territory Daman an...   
1  Malaika Arora slammed an Instagram user who tr...   
2  The Indira Gandhi Institute of Medical Science...   
3  Lashkar-e-Taiba's Kashmir commander Abu Dujana...   
4  Hotels in Maharashtra will train their staff t...   

                                               ctext  
0  summarize: The Daman and Diu administration on...  
1  summarize: From her special numbers to TV?appe...  
2  summarize: The Indira Gandhi Institute of Medi...  
3  summarize: Lashkar-e-Taiba's Kashmir commander...  
4  summarize: Hotels in Mumbai and other Indian c...  


In [21]:
# Creation of Dataset and Dataloader
# Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state = SEED)
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


In [22]:
# Creating the Training and Validation dataset for further creation of Dataloader
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
# Defining the parameters for creation of dataloaders
train_params = {
    'batch_size': TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }
val_params = {
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

In [23]:
# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

In [6]:
# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)
# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [10]:
!ls -lh

total 1.9M
-rwxrwxrwx 1 mosaic-ai mosaic-ai  302 Apr 26 05:21 '=0.11.1'
drwxrwxrwx 2 mosaic-ai mosaic-ai 4.0K Apr 26 05:21  GPT_pretrained
-rwxrwxrwx 1 mosaic-ai mosaic-ai   13 Apr 26 05:21  README.md
-rwxrwxrwx 1 mosaic-ai mosaic-ai  97K Apr 26 05:32  T5.ipynb
-rwxrwxrwx 1 mosaic-ai mosaic-ai  91K Apr 26 05:21  Untitled-Copy1.ipynb
-rwxrwxrwx 1 mosaic-ai mosaic-ai 1.6M Apr 26 05:21  Untitled.ipynb
-rwxrwxrwx 1 mosaic-ai mosaic-ai  590 Apr 26 05:21  Untitled1.ipynb
-rwxrwxrwx 1 mosaic-ai mosaic-ai  27K Apr 26 05:21  codeT5.ipynb
drwxrwxrwx 3 mosaic-ai mosaic-ai 4.0K Apr 26 05:21  drive
-rwxrwxrwx 1 mosaic-ai mosaic-ai   65 Apr 26 05:21  kaggle.json
drwxrwxrwx 2 mosaic-ai mosaic-ai 4.0K Apr 26 05:21  smart-app
-rwxrwxrwx 1 mosaic-ai mosaic-ai 4.0K Apr 26 05:21  yolopanda.ipynb


In [8]:
import os
filename = '/data/artifacts/pre-trained-T5/'
os.makedirs(os.path.dirname(filename), exist_ok=True)

tokenizer.save_pretrained(filename + "tokenizer/")
model.save_pretrained(filename + 'model/')

OSError: [Errno 30] Read-only file system: '/data/artifacts/pre-trained-T5'

In [25]:
# Training loop
print('Initiating Fine-Tuning for the model on our dataset')
for epoch in range(TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/packages/Python/c3e3a6d1-48c5-4ec2-8739-9b9b511c7036/3.8/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'Training Loss': 8.058304786682129}
Epoch: 0, Loss:  8.058304786682129
{'Training Loss': 2.1611168384552}
{'Training Loss': 3.1136369705200195}
{'Training Loss': 1.859143614768982}
{'Training Loss': 2.4855923652648926}
{'Training Loss': 2.607058525085449}
{'Training Loss': 3.0777788162231445}
{'Training Loss': 2.5979037284851074}
{'Training Loss': 1.6971368789672852}
{'Training Loss': 2.125770330429077}
{'Training Loss': 3.4051673412323}
{'Training Loss': 2.5892767906188965}
{'Training Loss': 1.6889514923095703}
{'Training Loss': 2.7192437648773193}
{'Training Loss': 2.3393096923828125}
{'Training Loss': 1.426955223083496}
{'Training Loss': 2.039098024368286}
{'Training Loss': 1.9689128398895264}
{'Training Loss': 2.7368674278259277}
{'Training Loss': 2.252192258834839}
{'Training Loss': 1.9333751201629639}
{'Training Loss': 1.7331931591033936}
{'Training Loss': 1.182373285293579}
{'Training Loss': 2.7537801265716553}
{'Training Loss': 1.0378023386001587}
{'Training Loss': 0.960289895

{'Training Loss': 1.325231671333313}
{'Training Loss': 1.568060278892517}
{'Training Loss': 1.0346990823745728}
{'Training Loss': 1.2913224697113037}
{'Training Loss': 1.017831802368164}
{'Training Loss': 2.127565383911133}
{'Training Loss': 1.5928393602371216}
{'Training Loss': 1.4604525566101074}
{'Training Loss': 1.3353371620178223}
{'Training Loss': 1.9908702373504639}
{'Training Loss': 1.0247286558151245}
{'Training Loss': 2.513697862625122}
{'Training Loss': 1.639871597290039}
{'Training Loss': 1.595216155052185}
{'Training Loss': 1.6392605304718018}
{'Training Loss': 0.9692506194114685}
{'Training Loss': 1.208333969116211}
{'Training Loss': 1.07382333278656}
Epoch: 1, Loss:  1.07382333278656
{'Training Loss': 1.3480194807052612}
{'Training Loss': 2.035156488418579}
{'Training Loss': 0.9873034358024597}
{'Training Loss': 1.032651424407959}
{'Training Loss': 0.8308926224708557}
{'Training Loss': 1.8320159912109375}
{'Training Loss': 1.2252055406570435}
{'Training Loss': 2.88793802

In [26]:
artifacts = (tokenizer, model, device, val_loader)

In [27]:
import pickle
import os
filename = '/data/artifacts/training_artifacts_T5_news1.pkl'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'wb') as f:
    pickle.dump(artifacts, f)

In [28]:
from datetime import datetime
print("Good morning... hope the code has run successfully... the time training was started : 2023-04-02 19:19:22. And the time it ended : "+ str(datetime.now()))

TypeError: bad operand type for unary +: 'str'

In [13]:
# model = T5ForConditionalGeneration.from_pretrained("t5-base")
# tokenizer = T5Tokenizer.from_pretrained("t5-base")
# !ls /home/mosaic-ai/.cache/huggingface/hub/models--t5-base/snapshots/
!mv /home/mosaic-ai/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef /home/mosaic-ai/.cache/huggingface/hub/models--t5-base/snapshots/0db7e623bcaee2daf9b859a646637ea39bf016cd

In [4]:
import pickle
model_loaded = T5ForConditionalGeneration.from_pretrained("/data/artifacts/model/")
tokenizer = T5Tokenizer.from_pretrained("/data/artifacts/tokenizer/")

In [16]:
# load the model from disk
filename = '/data/artifacts/training_artifacts_T5_news.pkl'
artifacts = pickle.load(open(filename, 'rb'))
print(type(artifacts))

<class 'tuple'>


In [17]:
artifacts

(T5Tokenizer(name_or_path='t5-base', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_4

In [18]:

# artifacts = (tokenizer, model, device, val_loader)
tokenizer = artifacts[0]
model = artifacts[1]
device = artifacts[2]
val_loader = artifacts[3]

In [ ]:
tokenizer.save_pretrained("/data/artifacts/tokenizer/")

In [ ]:
model.save_pretrained("/data/artifacts/model/")

In [ ]:
# Validation loop and saving the resulting file with predictions and acutals in a dataframe.
# Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv('/data/artifacts/predictions_artifacts_T5_news.csv')
    print('Output Files generated for review')

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe


/packages/Python/c3e3a6d1-48c5-4ec2-8739-9b9b511c7036/3.8/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Completed 0
Completed 100


In [5]:
mod= (tokenizer,model_loaded)

In [11]:
from refractml import *
from refractml.constants import MLModelFlavours

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
@scoring_func
def score(model, request):
    print("++++++++++++++++++---In Score Function---++++++++++++++++++++++")
    import pickle
    from transformers import T5Tokenizer, T5ForConditionalGeneration
    model_loaded = T5ForConditionalGeneration.from_pretrained("/data/artifacts/model/")
    print("++++++++++++++++++---Fine-Tuned Model Loaded---++++++++++++++++++++++")
    tokenizer = T5Tokenizer.from_pretrained("/data/artifacts/tokenizer/")
    print("++++++++++++++++++---Tokenizer Loaded---++++++++++++++++++++++")
    text = request.json["payload"]
#     tokenizer = model[0]
#     mod = model[1]
    print("++++++++++++++++++---Input Text Below---++++++++++++++++++++++")
    print(text)
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    mod = model_loaded
    source = tokenizer.batch_encode_plus([text], max_length=512, pad_to_max_length=True,return_tensors='pt')
    source_ids = source['input_ids']
    source_mask = source['attention_mask']
    print("++++++++++++++++++---Input Tokenized---++++++++++++++++++++++")
    generated_ids = mod.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=150, 
        num_beams=2,
        repetition_penalty=2.5, 
        length_penalty=1.0, 
        early_stopping=True
        
    )
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    print("++++++++++++++++++---Prediction Done, Ouput Below---++++++++++++++++++++++")
    
    print(preds)
    
    return preds


In [6]:
import requests
req = requests.Request()
req.json = {"payload":""""
New York City Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic.

During a press conference on Thursday, de Blasio said that the city will lift all restrictions on businesses, restaurants, and other venues starting on July 1st, as long as the vaccination rates continue to climb and the number of new COVID-19 cases remains low.

"This is going to be the summer of New York City," de Blasio said. "We are ready to bring back the life and energy that we all know and love in this city."

New York City was once the epicenter of the COVID-19 pandemic in the United States, with tens of thousands of cases and deaths in the early months of the outbreak. But the city has since made tremendous progress in controlling the spread of the virus, thanks in large part to an aggressive vaccination campaign that has already reached more than 6 million doses administered.

The city's seven-day average positivity rate is currently at 3.18%, down from a high of 7.39% in early January. The number of hospitalizations has also dropped significantly, with just over 2,000 patients currently being treated for COVID-19, down from a peak of more than 18,000 in April 2020.

The news of the city's reopening plan was met with enthusiasm from residents and business owners alike, many of whom have struggled to make ends meet during the pandemic.

"We've been waiting for this day for so long," said Lisa Ruiz, the owner of a small restaurant in Brooklyn. "It's been a tough year for us, but we're ready to come back stronger than ever."

However, some public health experts have expressed caution about the city's plans to fully reopen, noting that the virus is still present in the community and that new variants could pose a threat in the future.

"It's important that we continue to monitor the situation closely and be prepared to take action if necessary," said Dr. Jay Varma, the city's senior health advisor.

Despite these concerns, de Blasio said that he was confident that the city's vaccination efforts and other safety measures would be enough to prevent any major outbreaks.

"We're not going to take any chances, we're going to make sure that we're following the science and the data," he said. "But I believe that we can do this safely and responsibly, and we're going to make it happen."

The announcement of New York City's reopening plan comes as other parts of the country are also beginning to lift restrictions and return to normal. California, for example, announced last month that it would fully reopen on June 15th, while other states have already lifted mask mandates and capacity limits on businesses.

For many people, the news of the city's reopening is a sign of hope and optimism after a long and difficult year.

"I can't wait to go to concerts, restaurants, and all the other things that make New York City so special," said Paul Davis, a resident of the city. It's been a tough year, but this gives us something to look forward to."""}

In [8]:
response = score(None,req)

++++++++++++++++++---In Score Function---++++++++++++++++++++++
++++++++++++++++++---Fine-Tuned Model Loaded---++++++++++++++++++++++


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


++++++++++++++++++---Tokenizer Loaded---++++++++++++++++++++++
++++++++++++++++++---Input Text Below---++++++++++++++++++++++
"
New York City Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic.

During a press conference on Thursday, de Blasio said that the city will lift all restrictions on businesses, restaurants, and other venues starting on July 1st, as long as the vaccination rates continue to climb and the number of new COVID-19 cases remains low.

"This is going to be the summer of New York City," de Blasio said. "We are ready to bring back the life and energy that we all know and love in this city."

New York City was once the epicenter of the COVID-19 pandemic in the United States, with tens of thousands of cases and deaths in the early months of the outbreak. But the city has since made tremendous progress in controlling the spread of the virus, thanks in large part to an agg

In [9]:
response

['Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic. "We are ready to bring back the life and energy that we all know and love in this city," said Lisa Ruiz, the owner of a small restaurant in Brooklyn. However, public health experts have expressed caution about the city\'s plans to fully reopen.']

In [14]:
register_model((model,tokenizer),
               score,
               "T5_Summarization_v3",
               "T5_Summarization",
               MLModelFlavours.sklearn,
               init_script="pip install SentencePiece \\n pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"
              )

In [3]:
!python -m pip install refractml

  Using cached https://files.pythonhosted.org/packages/37/19/a4e26eaa2ef35252149e815df6766634185b4f8630e54c357b20cfae4ec9/refractml-1.0.2-py2.py3-none-any.whl
Processing /home/mosaic-ai/.cache/pip/wheels/ab/d0/0e/613976a1b51b5654859e2a82ade64329859bce431e280f2a39/shutils-0.1.0-cp38-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e7/e3/898487e5dbeb612054cf2e0c188463acb358167fef749c53c8bb8918cea1/cloudpickle-1.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/ef/7681134338fc097acef8d9b2f8abe0458e4d87559c689a8c306d0957ece5/requests_toolbelt-0.9.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/42/7ad4b6d67a16229496d4f6e74201bdbebcf4bc1e87d5a70c9297d4961bd2/PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/09/d7/8424b1dcaa5b1a2f824fc440aa1c4ef45e0bf6593d11b37962311614f365/mosaic_utils-1.0.2-py2.py3-no

You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
{"payload":"New York City Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic. During a press conference on Thursday, de Blasio said that the city will lift all restrictions on businesses, restaurants, and other venues starting on July 1st, as long as the vaccination rates continue to climb and the number of new COVID-19 cases remains low. 'This is going to be the summer of New York City,' de Blasio said. 'We are ready to bring back the life and energy that we all know and love in this city.' New York City was once the epicenter of the COVID-19 pandemic in the United States, with tens of thousands of cases and deaths in the early months of the outbreak. But the city has since made tremendous progress in controlling the spread of the virus, thanks in large part to an aggressive vaccination campaign that has already reached more than 6 million doses administered. The city's seven-day average positivity rate is currently at 3.18%, down from a high of 7.39% in early January. The number of hospitalizations has also dropped significantly, with just over 2,000 patients currently being treated for COVID-19, down from a peak of more than 18,000 in April 2020. The news of the city's reopening plan was met with enthusiasm from residents and business owners alike, many of whom have struggled to make ends meet during the pandemic. 'We have been waiting for this day for so long,' said Lisa Ruiz, the owner of a small restaurant in Brooklyn. 'It has been a tough year for us, but we're ready to come back stronger than ever.' However, some public health experts have expressed caution about the city's plans to fully reopen, noting that the virus is still present in the community and that new variants could pose a threat in the future. 'It is important that we continue to monitor the situation closely and be prepared to take action if necessary,i said Dr. Jay Varma, the city's senior health advisor. Despite these concerns, de Blasio said that he was confident that the city's vaccination efforts and other safety measures would be enough to prevent any major outbreaks. 'We are not going to take any chances, we're going to make sure that we're following the science and the data,' he said. 'But I believe that we can do this safely and responsibly, and we're going to make it happen.' The announcement of New York City's reopening plan comes as other parts of the country are also beginning to lift restrictions and return to normal. California, for example, announced last month that it would fully reopen on June 15th, while other states have already lifted mask mandates and capacity limits on businesses. For many people, the news of the city's reopening is a sign of hope and optimism after a long and difficult year. 'I can't wait to go to concerts, restaurants, and all the other things that make New York City so special,' said Paul Davis, a resident of the city. It's been a tough year, but this gives us something to look forward to."}

{'payload': "New York City Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic. During a press conference on Thursday, de Blasio said that the city will lift all restrictions on businesses, restaurants, and other venues starting on July 1st, as long as the vaccination rates continue to climb and the number of new COVID-19 cases remains low. 'This is going to be the summer of New York City,' de Blasio said. 'We are ready to bring back the life and energy that we all know and love in this city.' New York City was once the epicenter of the COVID-19 pandemic in the United States, with tens of thousands of cases and deaths in the early months of the outbreak. But the city has since made tremendous progress in controlling the spread of the virus, thanks in large part to an aggressive vaccination campaign that has already reached more than 6 million doses administered. The city's seven-day ave

In [15]:
{"data":["Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic. \"We are ready to bring back the life and energy that we all know and love in this city,\" said an owner of a small restaurant in Brooklyn. However, public health experts have expressed caution about the city's plans to fully reopen."],"request_id":"b38f7391-32fc-4a3b-99c6-bef4338d9671","upload_logging_data":{"end_time":"2023-04-24 05:30:00.963129","feedback":"","model_id":"636f1810-7f82-4cc5-81e4-c6ae4b0ffedc","request_id":"b38f7391-32fc-4a3b-99c6-bef4338d9671","response_data":["Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic. \"We are ready to bring back the life and energy that we all know and love in this city,\" said an owner of a small restaurant in Brooklyn. However, public health experts have expressed caution about the city's plans to fully reopen."],"start_time":"2023-04-24 05:29:30.218434","status":"Success","status_code":200,"version_id":"7e3864cc-852f-47de-b677-bd3251d46715"}}

{'data': ['Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic. "We are ready to bring back the life and energy that we all know and love in this city," said an owner of a small restaurant in Brooklyn. However, public health experts have expressed caution about the city\'s plans to fully reopen.'],
 'request_id': 'b38f7391-32fc-4a3b-99c6-bef4338d9671',
 'upload_logging_data': {'end_time': '2023-04-24 05:30:00.963129',
  'feedback': '',
  'model_id': '636f1810-7f82-4cc5-81e4-c6ae4b0ffedc',
  'request_id': 'b38f7391-32fc-4a3b-99c6-bef4338d9671',
  'response_data': ['Mayor Bill de Blasio has announced that the city plans to fully reopen on July 1st, marking a major milestone in the fight against the COVID-19 pandemic. "We are ready to bring back the life and energy that we all know and love in this city," said an owner of a small restaurant in Brooklyn. However, public health experts have 